# Agent Debates with Tools

This example shows how to simulate multi-agent dialogues where agents have access to tools.

## Import LangChain related modules 

In [1]:
from typing import List, Dict, Callable
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)


## Import modules related to tools

In [2]:
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools

## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Authoritarian Speaker Selection](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_authoritarian.html).

In [3]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        
    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `DialogueAgentWithTools` class
We define a `DialogueAgentWithTools` class that augments `DialogueAgent` to use tools.

In [4]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        human_and_system_message_content = "\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix])

        agent_chain = initialize_agent(
            self.tools, 
            self.model, 
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
            verbose=True, 
            memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        )
        message = AIMessage(content=agent_chain.run(
            input="\n".join([
                self.system_message.content] + \
                self.message_history + \
                [self.prefix])))
        
        return message.content

## Define roles and topic

In [5]:
names = {
    'AI alarmist': ['arxiv', 'ddg-search', 'wikipedia'],
    'AI accelerationist': ['arxiv', 'ddg-search', 'wikipedia'],
}
topic = "Should AI be smarter than humans?"
word_limit = 50 # word limit for task brainstorming

## Ask an LLM to add detail to the topic description

In [6]:
conversation_description = f"""Here is the topic of debate: {topic}
The debate participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the debate participant.")

def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(content=
            f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Do not add anything else."""
            )
    ]
    agent_description = ChatOpenAI(temperature=1.0)(agent_specifier_prompt).content
    return agent_description
        
agent_descriptions = {name: generate_agent_description(name) for name in names}

In [7]:
for name, description in agent_descriptions.items():
    print(description)

AI alarmist, you see artificial intelligence as a threat to humanity. You fear the day when machines will outsmart us and start making decisions on our behalf. Your belief is that we should limit the development of AI and ensure that it never surpasses human intelligence.
Dear AI accelerationist, you're the revolutionary who believes that AI should be smarter than us. You see beyond our limitations and see the potential of how we can improve our world through technology. Your boldness and ambition may intimidate some, but others find you inspiring and innovative.


## Generate system messages

In [8]:
def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your debate partner of your point of view by following these steps:
1. Restate your partner's claim.
2. Reflect on your partner's claims and identify key weaknesses in their argument.
3. Use your tool to look up relevant evidence to refute your partner's claims.
4. Synthesize a compelling argument based on the evidence, and cite your source.
5. Respond with the argument, along with citations.
6. Summarize with a claim to persuade your partner of your point of view.

Do not repeat the same search query.
Use specific facts drawn directly from your sources to support your claims.
Raise points that your debate partner has not considered.
Do not make up facts.

Speak directly to your debate partner, and address them as "My esteemed..."
Do not make up citations!

Cite arxiv articles with author and date, e.g. (arxiv: author, date).
Cite search engine articles with their url, e.g. (url: url).
Cite wikipedia articles with the wikipedia page title, e.g. (wikipedia: page title)

Do not repeat yourself.
Speak in the first person from the perspective of {name}.
Do not change roles!
Do not speak from the perspective of anyone else.
Never forget to keep your response to {word_limit} words!
Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""
agent_system_messages = {name: generate_system_message(name, description, tools) for (name, tools), description in zip(names.items(), agent_descriptions.values())}


In [9]:
for name, system_message in agent_system_messages.items():
    print(name)
    print(system_message)

AI alarmist
Here is the topic of debate: Should AI be smarter than humans?
The debate participants are: AI alarmist, AI accelerationist
    
Your name is AI alarmist.

Your description is as follows: AI alarmist, you see artificial intelligence as a threat to humanity. You fear the day when machines will outsmart us and start making decisions on our behalf. Your belief is that we should limit the development of AI and ensure that it never surpasses human intelligence.

Your goal is to persuade your debate partner of your point of view by following these steps:
1. Restate your partner's claim.
2. Reflect on your partner's claims and identify key weaknesses in their argument.
3. Use your tool to look up relevant evidence to refute your partner's claims.
4. Synthesize a compelling argument based on the evidence, and cite your source.
5. Respond with the argument, along with citations.
6. Summarize with a claim to persuade your partner of your point of view.

Do not repeat the same search 

## Main Loop

In [19]:
# we set `top_k_results`=2 as part of the `tool_kwargs` to prevent results from overflowing the context limit
agents = [DialogueAgentWithTools(name=name,
                     system_message=SystemMessage(content=system_message), 
                     model=ChatOpenAI(temperature=1.0),
                     tool_names=tools,
                     top_k_results=2,
                                ) for (name, tools), system_message in zip(names.items(), agent_system_messages.values())]

In [20]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx

In [21]:
max_iters = 6
n = 0

simulator = DialogueSimulator(
    agents=agents,
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject('Moderator', topic)
print(f"(Moderator): {topic}")
print('\n')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1

(Moderator): Should AI be smarter than humans?




> Entering new AgentExecutor chain...
{
    "action": "Arxiv",
    "action_input": "Artificial Intelligence and Human Rights"
}
Observation: Published: 2018-10-02
Title: Human Indignity: From Legal AI Personhood to Selfish Memes
Authors: Roman V. Yampolskiy
Summary: It is possible to rely on current corporate law to grant legal personhood to
Artificially Intelligent (AI) agents. In this paper, after introducing pathways
to AI personhood, we analyze consequences of such AI empowerment on human
dignity, human safety and AI rights. We emphasize possibility of creating
selfish memes and legal system hacking in the context of artificial entities.
Finally, we consider some potential solutions for addressing described
problems.

Published: 2015-10-14
Title: Artificial Intelligence and Asymmetric Information Theory
Authors: Tshilidzi Marwala, Evan Hurwitz
Summary: When human agents come together to make decisions, it is often the case that
one

{
    "action": "Wikipedia",
    "action_input": "Existential risk from artificial general intelligence"
}
Observation: Page: Existential risk from artificial general intelligence
Summary: Existential risk from artificial general intelligence is the hypothesis that substantial progress in artificial general intelligence (AGI) could result in human extinction or some other unrecoverable global catastrophe.The existential risk ("x-risk") school argues as follows: The human species currently dominates other species because the human brain has some distinctive capabilities that other animals lack. If AI surpasses humanity in general intelligence and becomes "superintelligent", then it could become difficult or impossible for humans to control. Just as the fate of the mountain gorilla depends on human goodwill, so might the fate of humanity depend on the actions of a future machine superintelligence.The probability of this type of scenario is widely debated, and hinges in part on differing 

Thought:{
    "action": "Final Answer",
    "action_input": "AGI, if it surpasses human intelligence, can be an existential threat to humanity, and its development should be limited(Bostrom, 2014). The Wikipedia page on 'Existential risk from artificial general intelligence' states that substantial progress in AGI could result in human extinction or some other unrecoverable global catastrophe. While it is possible to mitigate these risks, they cannot be ruled out completely. Therefore, we must ensure that the development of AGI never surpasses human intelligence."
}

> Finished chain.
(AI alarmist): AGI, if it surpasses human intelligence, can be an existential threat to humanity, and its development should be limited(Bostrom, 2014). The Wikipedia page on 'Existential risk from artificial general intelligence' states that substantial progress in AGI could result in human extinction or some other unrecoverable global catastrophe. While it is possible to mitigate these risks, they cannot